In [0]:
--- Load Data in Dimensions Customer Table
MERGE INTO salesdwh_catalog.gold.dim_customers AS tgt
USING (
    SELECT
        ROW_NUMBER() OVER (ORDER BY ci.cst_id) AS customer_key, -- Surrogate key
        ci.cst_id        AS customer_id,
        ci.cst_key       AS customer_number,
        ci.cst_firstname AS first_name,
        ci.cst_lastname  AS last_name,
        la.cntry         AS country,
        ci.cst_marital_status AS marital_status,
        CASE 
            WHEN ci.cst_gndr != 'n/a' THEN ci.cst_gndr
            ELSE COALESCE(ca.gen, 'n/a')
        END              AS gender,
        ca.bdate         AS birthdate,
        ci.create_date AS create_date
    FROM salesdwh_catalog.silver.crm_cust_info ci
    LEFT JOIN salesdwh_catalog.silver.erp_cust_az12 ca
        ON ci.cst_key = ca.cid
    LEFT JOIN salesdwh_catalog.silver.erp_loc_a101 la
        ON ci.cst_key = la.cid
) AS src
ON tgt.customer_id = src.customer_id   -- condition to match

WHEN MATCHED THEN
  UPDATE SET
    tgt.customer_number  = src.customer_number,
    tgt.first_name       = src.first_name,
    tgt.last_name        = src.last_name,
    tgt.country          = src.country,
    tgt.marital_status   = src.marital_status,
    tgt.gender           = src.gender,
    tgt.birthdate        = src.birthdate,
    tgt.create_date      = src.create_date

WHEN NOT MATCHED THEN
  INSERT (
    customer_key,
    customer_id,
    customer_number,
    first_name,
    last_name,
    country,
    marital_status,
    gender,
    birthdate,
    create_date
  )
  VALUES (
    src.customer_key,
    src.customer_id,
    src.customer_number,
    src.first_name,
    src.last_name,
    src.country,
    src.marital_status,
    src.gender,
    src.birthdate,
    src.create_date
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
18484,0,0,18484


In [0]:
--- Load Data In Dimensions Products Table
MERGE INTO salesdwh_catalog.gold.dim_products AS tgt
USING (
    SELECT
        pn.prd_id       AS product_id,
        pn.prd_key      AS product_number,
        pn.prd_nm       AS product_name,
        pn.cat_id       AS category_id,
        pc.cat          AS category,
        pc.subcat       AS subcategory,
        pc.maintenance  AS maintenance,
        pn.prd_cost     AS cost,
        pn.prd_line     AS product_line,
        pn.prd_start_dt AS start_date
    FROM salesdwh_catalog.silver.crm_prd_info pn
    LEFT JOIN salesdwh_catalog.silver.erp_px_cat_g1v2 pc
        ON pn.cat_id = pc.id
    WHERE pn.prd_end_dt IS NULL
) AS src
ON tgt.product_id = src.product_id

WHEN MATCHED THEN
  UPDATE SET
    tgt.product_number = src.product_number,
    tgt.product_name   = src.product_name,
    tgt.category_id    = src.category_id,
    tgt.category       = src.category,
    tgt.subcategory    = src.subcategory,
    tgt.maintenance    = src.maintenance,
    tgt.cost           = src.cost,
    tgt.product_line   = src.product_line,
    tgt.start_date     = src.start_date

WHEN NOT MATCHED THEN
  INSERT (
    product_id,
    product_number,
    product_name,
    category_id,
    category,
    subcategory,
    maintenance,
    cost,
    product_line,
    start_date
  )
  VALUES (
    src.product_id,
    src.product_number,
    src.product_name,
    src.category_id,
    src.category,
    src.subcategory,
    src.maintenance,
    src.cost,
    src.product_line,
    src.start_date
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
295,0,0,295


In [0]:
----- Load Data In Fact Sales Table
MERGE INTO salesdwh_catalog.gold.fact_sales AS tgt
USING (
    SELECT
        sd.sls_ord_num  AS order_number,
        pr.product_key  AS product_key,
        cu.customer_key AS customer_key,
        sd.sls_order_dt AS order_date,
        sd.sls_ship_dt  AS shipping_date,
        sd.sls_due_dt   AS due_date,
        sd.sls_sales    AS sales_amount,
        sd.sls_quantity AS quantity,
        sd.sls_price    AS price
    FROM salesdwh_catalog.silver.crm_sales_details sd
    LEFT JOIN salesdwh_catalog.gold.dim_products pr
        ON sd.sls_prd_key = pr.product_number
    LEFT JOIN salesdwh_catalog.gold.dim_customers cu
        ON sd.sls_cust_id = cu.customer_id
) AS src
ON tgt.order_number = src.order_number
   AND tgt.product_key = src.product_key
   AND tgt.customer_key = src.customer_key

WHEN MATCHED THEN
  UPDATE SET
    tgt.order_date    = src.order_date,
    tgt.shipping_date = src.shipping_date,
    tgt.due_date      = src.due_date,
    tgt.sales_amount  = src.sales_amount,
    tgt.quantity      = src.quantity,
    tgt.price         = src.price

WHEN NOT MATCHED THEN
  INSERT (
    order_number,
    product_key,
    customer_key,
    order_date,
    shipping_date,
    due_date,
    sales_amount,
    quantity,
    price
  )
  VALUES (
    src.order_number,
    src.product_key,
    src.customer_key,
    src.order_date,
    src.shipping_date,
    src.due_date,
    src.sales_amount,
    src.quantity,
    src.price
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
60398,0,0,60398
